In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
data = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
data.head()


In [ ]:
print(data["Class"].value_counts())

In [ ]:
data.info()


In [ ]:
from sklearn.preprocessing import StandardScaler

data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

In [ ]:
data = data.drop(['Time','Amount'],axis=1)
data.head()

In [ ]:
#下采样
X = data.iloc[:, data.columns != 'Class']
y = data.iloc[:, data.columns == 'Class']

# 得到所有异常样本的索引
number_records_fraud = len(data[data.Class == 1])
fraud_indices = np.array(data[data.Class == 1].index)

# 得到所有正常样本的索引
normal_indices = data[data.Class == 0].index

# 在正常样本中随机采样出指定个数的样本，并取其索引
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# 有了正常和异常样本后把它们的索引都拿到手
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# 根据索引得到下采样所有样本点
under_sample_data = data.iloc[under_sample_indices,:]

X_undersample = under_sample_data.iloc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.iloc[:, under_sample_data.columns == 'Class']

# 下采样 样本比例
print("正常样本所占整体比例: ", len(under_sample_data[under_sample_data.Class == 0])/len(under_sample_data))
print("异常样本所占整体比例: ", len(under_sample_data[under_sample_data.Class == 1])/len(under_sample_data))
print("下采样策略总体样本数量: ", len(under_sample_data))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
g = sns.FacetGrid(under_sample_data, col='Class')
g.map(plt.hist, 'V1', bins=20)

In [ ]:
g = sns.FacetGrid(under_sample_data, col='Class')
g.map(plt.hist, 'V9', bins=20) #Cabin

In [ ]:
from sklearn.model_selection import train_test_split

# 整个数据集进行划分
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

print("原始训练集包含样本数量: ", len(X_train))
print("原始测试集包含样本数量: ", len(X_test))
print("原始样本总数: ", len(X_train)+len(X_test))

# 下采样数据集进行划分
X_train_undersample, X_test_undersample, y_train_undersample, y_test_undersample = train_test_split(X_undersample
                                                                                                   ,y_undersample
                                                                                                   ,test_size = 0.2
                                                                                                   ,random_state = 0)
print("")
print("下采样训练集包含样本数量: ", len(X_train_undersample))
print("下采样测试集包含样本数量: ", len(X_test_undersample))
print("下采样样本总数: ", len(X_train_undersample)+len(X_test_undersample))

In [ ]:
from imblearn.over_sampling import SMOTE
oversampler=SMOTE(random_state=0)
os_features,os_labels=oversampler.fit_sample(X_train,y_train)
os_features = pd.DataFrame(os_features)
os_labels = pd.DataFrame(os_labels)
print(os_features.shape)
print(os_labels.shape)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.metrics import roc_auc_score, recall_score,precision_score
from time import time 
import datetime
from sklearn.tree import DecisionTreeClassifier
times=time()
#clf = DecisionTreeClassifier().fit(os_features,os_labels.values.ravel())
clf = LogisticRegression().fit(X_train,y_train.values.ravel())
result = clf.predict(X_test)
score = clf.score(X_test,y_test.values.ravel())
recall = recall_score(y_test.values.ravel(), result)
precision=precision_score(y_test.values.ravel(), result)
print("testing accuracy %f, recall is %f, precision is %f'" % (score,recall,precision))
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))


In [ ]:
from sklearn.metrics import roc_auc_score, recall_score,f1_score,precision_score
from time import time 
import datetime
clf = LogisticRegression(class_weight={1:11}).fit(X_train, y_train.values.ravel())
result = clf.predict(X_test)
score1 = clf.score(X_test,y_test.values.ravel())
recall1 = recall_score(y_test.values.ravel(), result)
precision1 = precision_score(y_test.values.ravel(), result)
f11=f1_score(y_test.values.ravel(), result)
print("testing accuracy %f, recall is %f,  f1 is %f ,  precision is %f" % (score1,recall1,f11,precision1))
#testing accuracy 0.999245, recall is 0.831683,  f1 is 0.796209

In [ ]:
from sklearn.metrics import roc_auc_score, recall_score
from time import time 
import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold
from sklearn.linear_model import LogisticRegression
#设置kfold，交叉采样法拆分数据集
kfold=StratifiedKFold(n_splits=10)
times=time()
solver = ['liblinear','lbfgs','newton-cg','sag','saga']#返回在对数刻度上均匀间隔的数字
range1=range(0,len(solver),1)
score = []
recall=[]
roc_auc=[]
precision=[]
f1=[]
for i in solver:
    print(i)
    clf = LogisticRegression(class_weight={1:11},solver=i)
    recall.append(cross_val_score(clf,X_train,y_train.values.ravel(),
                                          scoring='recall',cv=kfold,n_jobs=-1).mean())
    f1.append(cross_val_score(clf,X_train,y_train.values.ravel(),
                                          scoring='f1',cv=kfold,n_jobs=-1).mean()) 
    print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))

plt.plot(range1,recall,c="blue",label="recall")
plt.plot(range1,f1,c="yellow",label="f1")
plt.legend(fontsize="x-large")
plt.show()
print(datetime.datetime.fromtimestamp(time()-times).strftime("%M:%S:%f"))


In [ ]:
from sklearn.metrics import confusion_matrix,recall_score
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
matrix = confusion_matrix(y_test, result,labels=[1,0])
print("混淆矩阵:\n", matrix)
print("精度:", precision_score(y_test, result))
print("召回率:", recall_score(y_test, result))
print("f1分数:", f1_score(y_test,result))
print("准确率",accuracy_score(y_test, result))


In [ ]:
clf.predict_proba(X_test)

In [ ]:
from sklearn.metrics import roc_curve as ROC
import matplotlib.pyplot as plt
FPR, Recall, thresholds = ROC(y_test,clf.predict_proba(X_test)[:,1],pos_label=1)
area = roc_auc_score(y_test,clf.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(FPR, Recall, color='red',
         label='ROC curve (area = %0.2f)' % area)
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('Recall')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()


In [ ]:
maxindex = (Recall - FPR).tolist().index(max(Recall - FPR))
aa=thresholds[maxindex]
print(aa)

clf = LogisticRegression(class_weight={1:11}).fit(X_train, y_train.values.ravel())
prob=clf.predict_proba(X_test)
prob=pd.DataFrame(prob)
prob.loc[prob.iloc[:,1] >= aa,"y_pred"]=1
prob.loc[prob.iloc[:,1] < aa,"y_pred"]=0
result=prob.loc[:,"y_pred"].values
matrix = confusion_matrix(y_test, result,labels=[1,0])
print("混淆矩阵:\n", matrix)
print("精度:", precision_score(y_test, result))
print("召回率:", recall_score(y_test, result))
print("f1分数:", f1_score(y_test,result))
print("准确率",accuracy_score(y_test, result))

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    绘制混淆矩阵
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import accuracy_score as AC
from sklearn.metrics import confusion_matrix,recall_score
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
clf = LogisticRegression(class_weight={1:11}).fit(X_train, y_train.values.ravel())
prob=clf.predict_proba(X_test)
prob=pd.DataFrame(prob)
#print(prob.head())
thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
j=1
plt.figure(figsize=(10,10))
for i in thresholds:
    prob.loc[prob.iloc[:,1] >= i,"y_pred"]=1
    prob.loc[prob.iloc[:,1] < i,"y_pred"]=0
    result=prob.loc[:,"y_pred"].values
    matrix = confusion_matrix(y_test, result,labels=[1,0])
    print(i)
    print("混淆矩阵:\n", matrix)
    print("精度:", precision_score(y_test, result))
    print("召回率:", recall_score(y_test, result))
    print("f1分数:", f1_score(y_test,result))
    print("准确率",accuracy_score(y_test, result))
    print("-----------")
    class_names = [1,0]
    plt.subplot(3,3,j)
    j=j+1
    
    plot_confusion_matrix(matrix
                          , classes=class_names
                          , title='Threshold >= %s'%i) 

